In [22]:
# Load modules
import os, sys
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
import utils as ut
import loss_funcs as lf

In [4]:
# Load data
df = pd.read_csv('../output/compas-scores-two-years_cleaned.csv')
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,Male,25 - 45,African-American,-0.733607,F,1,-0.167773
1,Male,< 25,African-American,0.055928,F,1,-0.340654
2,Male,25 - 45,Caucasian,2.029767,F,1,-0.244609
3,Female,25 - 45,Caucasian,-0.733607,M,0,-0.321445
4,Male,< 25,Caucasian,-0.536224,F,1,-0.359864


In [5]:
# Encoding to dummy variables
df['sex'] = df['sex'].apply(lambda sex: 0 if sex == 'Female' else 1)
df['age_cat'] = df['age_cat'].apply(lambda age_cat: 2 if age_cat == '> 45' else(1 if age_cat == '25 - 45' else 0))
df['race'] = df['race'].apply(lambda race: 0 if race == 'African-American' else 1)
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda c_charge_degree: 0 if c_charge_degree == 'M' else 1)
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,1,1,0,-0.733607,1,1,-0.167773
1,1,0,0,0.055928,1,1,-0.340654
2,1,1,1,2.029767,1,1,-0.244609
3,0,1,1,-0.733607,0,0,-0.321445
4,1,0,1,-0.536224,1,1,-0.359864


In [7]:
# Checking the mapping
df.isnull().sum()

sex                0
age_cat            0
race               0
priors_count       0
c_charge_degree    0
two_year_recid     0
length_of_stay     0
dtype: int64

In [11]:
# Storing features
features = ['sex', 'age_cat', 'priors_count', 'c_charge_degree', 'length_of_stay']
sensitive = 'race'
target = 'two_year_recid'

In [12]:
# Function to process and shuffle data
def preprocessing(df):
    y_label = df[target]
    protected_attribute = df[sensitive]
    df_new = df[features]
    y_label, protected_attribute, df_new = shuffle(y_label, protected_attribute, df_new, random_state = 704)
    
    return y_label.to_numpy(), protected_attribute.to_numpy(), df_new.to_numpy()


In [13]:
# Split data into train and test
y_label, protected_attribute, df_new =  preprocessing(df)
train_index = int(len(df_new) * 0.7)
x_train, y_train, race_train = df_new[:train_index], y_label[:train_index], protected_attribute[:train_index]
x_test, y_test, race_test = df_new[train_index:], y_label[train_index:],protected_attribute[train_index:]

In [14]:
# P-Rule function for evaluation
def p_rule(sensitive_var, y_pred):
    protected = np.where(sensitive_var == 1)[0]
    not_protected = np.where(sensitive_var == 0)[0]
    protected_pred = np.where(y_pred[protected] == 1)
    not_protected_pred = np.where(y_pred[not_protected] == 1)
    protected_percent = protected_pred[0].shape[0]/protected.shape[0]
    not_protected_percent = not_protected_pred[0].shape[0]/not_protected.shape[0]
    ratio = min(protected_percent/not_protected_percent, not_protected_percent/protected_percent)
    
    return ratio, protected_percent, not_protected_percent

The "80% rule" (or the p%-rule) is a guideline established by the U.S. Equal Employment Opportunity Commission (EEOC) to help identify potential discrimination in hiring, promotion, or other employment decisions. It's a way to measure fairness and equal opportunity in these processes, particularly concerning sensitive attributes such as race, gender, age, or disability.

In [15]:
# Calibration function for evaluation
def calibration(sensitive_var, y_pred, y_true):
    protected_point = np.where(sensitive_var == 1)[0]
    y_predcau = y_pred[protected_point]
    y_truecau = y_true[protected_point]
    pcau = sum(y_predcau==y_truecau)/len(y_truecau)
    not_protected_point = np.where(sensitive_var == 0)[0]
    y_predafa = y_pred[not_protected_point]
    y_trueafa = y_true[not_protected_point]
    pafa = sum(y_predafa==y_trueafa)/len(y_trueafa)
    calibration = abs(pcau-pafa)
    return(calibration)

### Training Unconstrained Classifier

In [16]:
# Train model and print results
clf = LogisticRegression(random_state = 0).fit(x_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))
results_lr = {"Classifier": ["LR", "LR"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [clf.score(x_train, y_train)*100, clf.score(x_test, y_test)*100],
              "P-rule (%)": [p_rule(race_train, clf.predict(x_train))[0]*100, p_rule(race_test, clf.predict(x_test))[0]*100],
              "Protected (%)": [p_rule(race_train, clf.predict(x_train))[1]*100, p_rule(race_test, clf.predict(x_test))[1]*100],
              "Not protected (%)": [p_rule(race_train, clf.predict(x_train))[2]*100, p_rule(race_test, clf.predict(x_test))[2]*100],
              "Calibration (%)": [calibration(race_train, clf.predict(x_train), y_train)*100, calibration(race_test, clf.predict(x_test), y_test)*100]}
pd.DataFrame(results_lr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,LR,Train,65.724638,57.224435,31.223881,54.563895,0.690624
1,LR,Test,66.985915,53.653654,29.729730,55.410448,0.450097


P-rule%: This metric assesses fairness in the classifier with respect to the protected and non-protected groups. A value closer to 100% indicates better fairness. The rule of thumb is 80%.

Calibration: the difference in prediction accuracy between protected and unprotected groups. It measures how well the model's prediction align with the true outcomes. The difference indicates whether the model is calibrated similiarly for protected and unprotected groups. 

Protected (%): The percentage of positive outcomes (e.g., being hired) in the protected group (Caucasians). This helps to understand how the classifier performs on the protected group. The ratio of the number of positive outcomes for Caucasians to the total number of Caucasians in the dataset.

Not protected (%): The percentage of positive outcomes in the non-protected group (African-American).  It is the ratio of the number of positive outcomes for African-Americans to the total number of African-Americans in the dataset.



### Interpretation:

The accuracy for the test data is higher (66.98%). Nonetheless, the p-rule for test and training data are less than 80% indicating there may be some bias in the classifier. In this case, the calibration values are 0.69% for the Train dataset and 0.45% for the Test dataset, indicating that the model has little differences in prediction accuracies for both groups. (TBC: What is the threshold?)

### To improve fairness, it might be necessary to investigate the cause of the lower p-rule ratio and consider adjusting the classifier or incorporating fairness-enhancing techniques


### Applying Fairness Contraints 

The fairness constraint is set to achieve a 0 covariance between the sensitive feature (race) and the distance to the decision boundary. A 0 covariance means there is no correlation between the two variables, which helps promote fairness.

1. Train the model using constraints to extract model weights (cweight). 

In [18]:
# Setting Constrains
#In this case, only fairness constraints are applied (apply_fairness_constraints = 1)
# While accuracy constraint and separate constraint are not applied (both set to 0).
fairness_constraint = 1 
accuracy_constraint = 0
separate_constraint = 0
gamma = None
sensitive_attrs = ['race']
sensitive_attrs_to_cov_thresh = {'race': 0}
x_control = {'race': race_train}


In [24]:
# Training model with constraints
np.random.seed(704)
cweight = ut.train_model(x_train,
                   y_train,
                   x_control,
                   lf._logistic_loss,
                   fairness_constraint,
                   accuracy_constraint,
                   separate_constraint,
                   sensitive_attrs,
                   sensitive_attrs_to_cov_thresh,
                   gamma)

2. Feed the model with trained weights

In [25]:
# Feeding model with coefficients and weights
m = LogisticRegression()
m.coef_= cweight.reshape((1,-1))
m.intercept_ = 0
m.classes_ = np.array([0, 1])

3. Assess the result

In [26]:
# Print the results
results_clr = {"Classifier": ["C-LR", "C-LR"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [m.score(x_train, y_train)*100, m.score(x_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, m.predict(x_train))[0]*100, p_rule(race_test, m.predict(x_test))[0]*100],
               "Protected (%)": [p_rule(race_train, m.predict(x_train))[1]*100, p_rule(race_test, m.predict(x_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, m.predict(x_train))[2]*100, p_rule(race_test, m.predict(x_test))[2]*100],
               "Calibration (%)": [calibration(race_train, m.predict(x_train), y_train)*100, calibration(race_test, m.predict(x_test), y_test)*100]}
pd.DataFrame(results_clr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%),Calibration (%)
0,C-LR,Train,47.946860,99.942529,99.820896,99.878296,11.642275
1,C-LR,Test,47.267606,99.950990,99.857752,99.906716,15.142593


### Interpretation:

The accuracy is lower compared to the based-model. Fairness wise, p-rule is drastically improved nearing 100% indicating the model is fair vis-a-vis to race. Meanwhile, the calibration is quite high compared to based-model.